In [ ]:
from sys import exit
import numpy as np
import pandas as pd
from os.path import dirname, join, isfile, isdir
from os import makedirs
from datetime import datetime, timedelta, date

from bulletin.data.notifica import Notifica
from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.commom.utils import Timer, get_better_notifica
from bulletin.commom.static import meses
from bulletin.metabase.request import download_metabase

pd.set_option('display.max_columns', None)

output = join("output","correcoes","notifica")

if not isdir(output):
    makedirs(output)

timerAll = Timer()
timer = Timer()

In [ ]:
cc = CasosConfirmados()
# cc.read()
# cc.save()
cc.load()
notifica = Notifica()
# notifica.download_todas_notificacoes()
# notifica.save()
notifica.load()
print(cc.shape())
print(notifica.shape())

In [ ]:
casos_confirmados = cc.get_casos()
casos_notifica = notifica.get_casos()
print(casos_notifica.shape)

In [ ]:
merge_casos_confirmados = pd.merge(casos_confirmados,casos_notifica[['id','mun_resid']],how='left',on='mun_resid')
print(merge_casos_confirmados.shape)

In [ ]:
cc_sem_id = merge_casos_confirmados.loc[merge_casos_confirmados['id'].isnull()].copy()
cc_sem_id = cc_sem_id.drop(columns='id')

merge_casos_confirmados = merge_casos_confirmados.loc[merge_casos_confirmados['id'].notnull()]
id_sem_cc = casos_notifica.loc[~casos_notifica['id'].isin(merge_casos_confirmados['id'])]
###
merge_casos_confirmados = merge_casos_confirmados.append(pd.merge(cc_sem_id,id_sem_cc[['id','hash_atend']],how='left',on='hash_atend'))
print(merge_casos_confirmados.shape)

In [ ]:
cc_sem_id = merge_casos_confirmados.loc[merge_casos_confirmados['id'].isnull()].copy()
cc_sem_id = cc_sem_id.drop(columns='id')

merge_casos_confirmados = merge_casos_confirmados.loc[merge_casos_confirmados['id'].notnull()]
id_sem_cc = casos_notifica.loc[~casos_notifica['id'].isin(merge_casos_confirmados['id'])]
###
merge_casos_confirmados = merge_casos_confirmados.append(pd.merge(cc_sem_id,id_sem_cc[['id','hash_diag']],how='left',on='hash_diag'))
print(merge_casos_confirmados.shape)